In [1]:
#libraries

import pandas as pd
import numpy as np
import psycopg2 # needed to get database exception errors when uploading dataframe
import requests # package for getting data from the web
from zipfile import * # package for unzipping zip files
import sqlalchemy
import os

In [25]:
agri_land = pd.read_csv("kaggle/Agricultural Land.csv", index_col=0)
corps_units = pd.read_csv("kaggle/Crops_Units.csv", index_col=0)
fresh_gw = pd.read_csv("kaggle/Fresh groundwater abstracted.csv", index_col=0)
fresh_ab = pd.read_csv("kaggle/Freshwater abstracted.csv", index_col=0)
ghg = pd.read_csv("kaggle/GHG_Emissions_by_Sector.csv", index_col=0)
prec = pd.read_csv("Precipitation.csv", index_col=0)

In [3]:
prec

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,Footnotes\n2008,Footnotes\n2009,Footnotes\n2010,Footnotes\n2011,Footnotes\n2012,Footnotes\n2013,Footnotes\n2014,Footnotes\n2015,Footnotes\n2016,Footnotes\n2017
CountryID,,,,,,,,,,,,,,,,,,,,,
8.0,Albania,28385,...,...,...,...,40311,...,...,...,...,1.0,1.0,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0
12.0,Algeria,275681.3125,...,...,...,...,376971.5,194723.2969,217926,198905.2031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20.0,Andorra,471.7792969,...,...,...,...,424.8518982,625.1678467,444.7502136,361.2449341,...,3.0,3.0,3,3.0,3.0,4.0,4.0,4.0,NaN,NaN
660.0,Anguilla,93.09999847,...,...,...,...,100.7300034,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28.0,Antigua and Barbuda,300.2999878,264.3999939,475.7999878,322.5,283.7999878,374.5,323.2999878,279.2000122,384.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887.0,Yemen,68000,...,...,...,...,...,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
894.0,Zambia,...,...,...,...,...,...,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
716.0,Zimbabwe,244262.2031,...,...,...,...,163649.0313,273803.4375,315223.6875,209523.9063,...,34.0,34.0,34,34.0,34.0,34.0,34.0,34.0,34.0,34.0


In [4]:
prec.isnull().sum()

Country              2
1990                 2
1991                 2
1992                 2
1993                 2
1994                 2
1995                 2
1996                 2
1997                 2
1998                 2
1999                 2
2000                 2
2001                 2
2002                 2
2003                 2
2004                 2
2005                 2
2006                 2
2007                 2
2008                 2
2009                 2
2010                 2
2011                 2
2012                 2
2013                 2
2014                 2
2015                 2
2016                 2
2017                 2
Footnotes\n1990    111
Footnotes\n1991    120
Footnotes\n1992    120
Footnotes\n1993    120
Footnotes\n1994    120
Footnotes\n1995    113
Footnotes\n1996    112
Footnotes\n1997    111
Footnotes\n1998    110
Footnotes\n1999    109
Footnotes\n2000    109
Footnotes\n2001    109
Footnotes\n2002    109
Footnotes\n2003    110
Footnotes\n

In [5]:
prec.columns

Index(['Country', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', 'Footnotes\n1990', 'Footnotes\n1991',
       'Footnotes\n1992', 'Footnotes\n1993', 'Footnotes\n1994',
       'Footnotes\n1995', 'Footnotes\n1996', 'Footnotes\n1997',
       'Footnotes\n1998', 'Footnotes\n1999', 'Footnotes\n2000',
       'Footnotes\n2001', 'Footnotes\n2002', 'Footnotes\n2003',
       'Footnotes\n2004', 'Footnotes\n2005', 'Footnotes\n2006',
       'Footnotes\n2007', 'Footnotes\n2008', 'Footnotes\n2009',
       'Footnotes\n2010', 'Footnotes\n2011', 'Footnotes\n2012',
       'Footnotes\n2013', 'Footnotes\n2014', 'Footnotes\n2015',
       'Footnotes\n2016', 'Footnotes\n2017'],
      dtype='object')

In [6]:
prec.drop(['Footnotes\n1990', 'Footnotes\n1991',
       'Footnotes\n1992', 'Footnotes\n1993', 'Footnotes\n1994',
       'Footnotes\n1995', 'Footnotes\n1996', 'Footnotes\n1997',
       'Footnotes\n1998', 'Footnotes\n1999', 'Footnotes\n2000',
       'Footnotes\n2001', 'Footnotes\n2002', 'Footnotes\n2003',
       'Footnotes\n2004', 'Footnotes\n2005', 'Footnotes\n2006',
       'Footnotes\n2007', 'Footnotes\n2008', 'Footnotes\n2009',
       'Footnotes\n2010', 'Footnotes\n2011', 'Footnotes\n2012',
       'Footnotes\n2013', 'Footnotes\n2014', 'Footnotes\n2015',
       'Footnotes\n2016', 'Footnotes\n2017'], axis='columns',
                                        inplace= True)

In [7]:
prec

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
CountryID,,,,,,,,,,,,,,,,,,,,,
8.0,Albania,28385,...,...,...,...,40311,...,...,...,...,29458,37218,51134,20569,43050,30615,37172,32711,38810,32224
12.0,Algeria,275681.3125,...,...,...,...,376971.5,194723.2969,217926,198905.2031,...,544728.625,313696.5,373310.0938,291255.5,285533.0938,276258.9063,237403,238029.5,181922.0938,330957.4063
20.0,Andorra,471.7792969,...,...,...,...,424.8518982,625.1678467,444.7502136,361.2449341,...,466.674469,464.0131226,423.6300049,355.0338745,352.9525146,482.6208496,551.6988525,474.7380066,...,...
660.0,Anguilla,93.09999847,...,...,...,...,100.7300034,...,...,...,...,96.88999939,71.08000183,...,...,...,...,...,...,...,...
28.0,Antigua and Barbuda,300.2999878,264.3999939,475.7999878,322.5,283.7999878,374.5,323.2999878,279.2000122,384.5,...,392.5,276.8999939,463.8999939,450.2000122,279.7000122,328.2000122,257.5,161.1000061,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887.0,Yemen,68000,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894.0,Zambia,...,...,...,...,...,...,...,...,...,...,1019.900024,1002.200012,1008.799988,890.4000244,1033.900024,...,...,...,...,...
716.0,Zimbabwe,244262.2031,...,...,...,...,163649.0313,273803.4375,315223.6875,209523.9063,...,311128.875,286966.0313,277085.7813,291270.2813,259853.4063,300531.2188,294826.1563,244809.2656,211907.5156,372899.4063


In [8]:
agri_land

,Agricultural area in 2013 (km2),% change of agricultural area since 1990,% of total land area covered by agricultural area in 2013,Arable land in 2013 (km2),Permanent crops in 2013 (km2),Permanent meadows and pastures in 2013 (km2),Agricultural area actually irrigated in 2013 (km2),Unnamed: 8
Country,,,,,,,,
Afghanistan,379 100,-0.3,58.1,77 850,1 250,300 000,20 920,NaN
Albania,11 873,5.9,43.3,6 171,792,4 910,2 053,NaN
Algeria,414 316,7.1,17.4,74 962,9 390,329 964,10 895,NaN
American Samoa,49,63.3,24.5,30,19,...,...,NaN
Andorra,208,9.5,44.3,29,...,179,...,NaN
...,...,...,...,...,...,...,...,...
Wallis and Futuna Islands,60,0.0,42.9,10,50,...,...,NaN
Western Sahara,50 040,0.0,18.8,40,...,50 000,...,NaN
Yemen,235 460,-0.3,44.6,12 480,2 980,220 000,...,NaN


In [9]:
agri_land.columns

Index(['Agricultural area in 2013 (km2)',
       '% change of agricultural area since 1990',
       '% of total land area covered by agricultural area in 2013',
       'Arable land in 2013 (km2)', 'Permanent crops in 2013 (km2)',
       'Permanent meadows and pastures in 2013 (km2)',
       'Agricultural area actually irrigated in 2013 (km2)', 'Unnamed: 8'],
      dtype='object')

In [10]:
agri_land.isnull().sum()

Agricultural area in 2013 (km2)                                0
% change of agricultural area since 1990                       0
% of total land area covered by agricultural area in 2013      0
Arable land in 2013 (km2)                                      0
Permanent crops in 2013 (km2)                                  0
Permanent meadows and pastures in 2013 (km2)                   0
Agricultural area actually irrigated in 2013 (km2)             0
Unnamed: 8                                                   225
dtype: int64

In [11]:
agri_land.drop('Unnamed: 8', axis='columns', inplace= True)

In [12]:
agri_land

,Agricultural area in 2013 (km2),% change of agricultural area since 1990,% of total land area covered by agricultural area in 2013,Arable land in 2013 (km2),Permanent crops in 2013 (km2),Permanent meadows and pastures in 2013 (km2),Agricultural area actually irrigated in 2013 (km2)
Country,,,,,,,
Afghanistan,379 100,-0.3,58.1,77 850,1 250,300 000,20 920
Albania,11 873,5.9,43.3,6 171,792,4 910,2 053
Algeria,414 316,7.1,17.4,74 962,9 390,329 964,10 895
American Samoa,49,63.3,24.5,30,19,...,...
Andorra,208,9.5,44.3,29,...,179,...
...,...,...,...,...,...,...,...
Wallis and Futuna Islands,60,0.0,42.9,10,50,...,...
Western Sahara,50 040,0.0,18.8,40,...,50 000,...
Yemen,235 460,-0.3,44.6,12 480,2 980,220 000,...


In [13]:
agri_land.agg('Agricultural area in 2013 (km2)')

Country
Afghanistan                    379 100
Albania                         11 873
Algeria                        414 316
American Samoa                      49
Andorra                            208
                               ...    
Wallis and Futuna Islands           60
Western Sahara                  50 040
Yemen                          235 460
Zambia                         237 360
Zimbabwe                       162 000
Name: Agricultural area in 2013 (km2), Length: 225, dtype: object

In [14]:
corps_units.describe()

,Description
count,60
unique,59
top,United States dollar
freq,2


In [15]:
fresh_gw

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,Footnotes 2008,Footnotes 2009,Footnotes 2010,Footnotes 2011,Footnotes 2012,Footnotes 2013,Footnotes 2014,Footnotes 2015,Footnotes 2016,Footnotes 2017
CountryID,,,,,,,,,,,,,,,,,,,,,
8.0,Albania,...,...,...,...,...,...,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12.0,Algeria,...,...,...,...,...,...,...,...,...,...,NaN,NaN,NaN,NaN,NaN,2,2.0,2.0,2.0,2.0
28.0,Antigua and Barbuda,1.139999986,...,...,...,...,0.550000012,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51.0,Armenia,1325.400024,...,...,...,...,851,616,520,530,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31.0,Azerbaijan,1706,...,...,...,...,1150,987,1098,681,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887.0,Yemen,...,...,...,...,...,2442,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
894.0,Zambia,...,...,...,...,...,286.5,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
716.0,Zimbabwe,9270.139648,...,...,...,...,7572.710449,10668.95801,11732.38574,10895.43945,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
fresh_gw.isnull().sum()

Country             2
1990                2
1991                2
1992                2
1993                2
1994                2
1995                2
1996                2
1997                2
1998                2
1999                2
2000                2
2001                2
2002                2
2003                2
2004                2
2005                2
2006                2
2007                2
2008                2
2009                2
2010                2
2011                2
2012                2
2013                2
2014                2
2015                2
2016                2
2017                2
Footnotes 1990    109
Footnotes 1991    114
Footnotes 1992    114
Footnotes 1993    114
Footnotes 1994    114
Footnotes 1995    110
Footnotes 1996    112
Footnotes 1997    112
Footnotes 1998    112
Footnotes 1999    110
Footnotes 2000    108
Footnotes 2001    106
Footnotes 2002    108
Footnotes 2003    108
Footnotes 2004    105
Footnotes 2005     97
Footnotes 

In [17]:
fresh_gw.columns

Index(['Country', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', 'Footnotes 1990', 'Footnotes 1991',
       'Footnotes 1992', 'Footnotes 1993', 'Footnotes 1994', 'Footnotes 1995',
       'Footnotes 1996', 'Footnotes 1997', 'Footnotes 1998', 'Footnotes 1999',
       'Footnotes 2000', 'Footnotes 2001', 'Footnotes 2002', 'Footnotes 2003',
       'Footnotes 2004', 'Footnotes 2005', 'Footnotes 2006', 'Footnotes 2007',
       'Footnotes 2008', 'Footnotes 2009', 'Footnotes 2010', 'Footnotes 2011',
       'Footnotes 2012', 'Footnotes 2013', 'Footnotes 2014', 'Footnotes 2015',
       'Footnotes 2016', 'Footnotes 2017'],
      dtype='object')

In [18]:
fresh_gw.drop(['Footnotes 1990', 'Footnotes 1991',
       'Footnotes 1992', 'Footnotes 1993', 'Footnotes 1994', 'Footnotes 1995',
       'Footnotes 1996', 'Footnotes 1997', 'Footnotes 1998', 'Footnotes 1999',
       'Footnotes 2000', 'Footnotes 2001', 'Footnotes 2002', 'Footnotes 2003',
       'Footnotes 2004', 'Footnotes 2005', 'Footnotes 2006', 'Footnotes 2007',
       'Footnotes 2008', 'Footnotes 2009', 'Footnotes 2010', 'Footnotes 2011',
       'Footnotes 2012', 'Footnotes 2013', 'Footnotes 2014', 'Footnotes 2015',
       'Footnotes 2016', 'Footnotes 2017'], axis='columns',
                                        inplace= True)

In [19]:
fresh_gw

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
CountryID,,,,,,,,,,,,,,,,,,,,,
8.0,Albania,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,235,262,...,107
12.0,Algeria,...,...,...,...,...,...,...,...,...,...,4150,4200,2603,2915,3000,3046.909912,3117.449951,3654.939941,3900,3860
28.0,Antigua and Barbuda,1.139999986,...,...,...,...,0.550000012,...,...,...,...,1.399999976,1.600000024,1.600000024,1.200000048,1.200000048,0.720000029,0.680000007,0.74000001,...,...
51.0,Armenia,1325.400024,...,...,...,...,851,616,520,530,...,852,854.4000244,875.7999878,1002.799988,1314.400024,1348.699951,1312,1304.400024,1136.300049,1154.5
31.0,Azerbaijan,1706,...,...,...,...,1150,987,1098,681,...,1127,1262,1272,1506,1696,1738,1819,2016,2075,2066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887.0,Yemen,...,...,...,...,...,2442,...,...,...,...,...,...,3805,...,...,...,...,4411,...,...
894.0,Zambia,...,...,...,...,...,286.5,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716.0,Zimbabwe,9270.139648,...,...,...,...,7572.710449,10668.95801,11732.38574,10895.43945,...,14684.69141,16608.80664,16810.83594,16906.18359,13721.11328,14798.3916,13818.21973,10370.45215,12507.52441,13714.00684


In [20]:
fresh_ab

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,Footnotes\r\n2008,Footnotes\r\n2009,Footnotes\r\n2010,Footnotes\r\n2011,Footnotes\r\n2012,Footnotes\r\n2013,Footnotes\r\n2014,Footnotes\r\n2015,Footnotes\r\n2016,Footnotes\r\n2017
CountryID,,,,,,,,,,,,,,,,,,,,,
8.0,Albania,...,...,...,...,...,...,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12.0,Algeria,...,...,...,...,...,...,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20.0,Andorra,...,...,...,...,...,...,...,...,...,...,2,2,2.0,2.0,2.0,2,2.0,2.0,NaN,NaN
28.0,Antigua and Barbuda,3.380000114,...,...,...,...,0.970000029,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51.0,Armenia,3942,...,...,...,...,2331,2077,2250.699951,2144,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887.0,Yemen,...,...,...,...,...,3942,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
894.0,Zambia,...,...,...,...,...,2431,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
716.0,Zimbabwe,33107.64063,...,...,...,...,27045.39453,38103.42188,41901.375,38912.28516,...,39,39,39.0,39.0,39.0,39,39.0,39.0,39.0,39.0


In [26]:
fresh_ab.columns

Index(['Country', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', 'Footnotes\r\n1990', 'Footnotes\r\n1991',
       'Footnotes\r\n1992', 'Footnotes\r\n1993', 'Footnotes\r\n1994',
       'Footnotes\r\n1995', 'Footnotes\r\n1996', 'Footnotes\r\n1997',
       'Footnotes\r\n1998', 'Footnotes\r\n1999', 'Footnotes\r\n2000',
       'Footnotes\r\n2001', 'Footnotes\r\n2002', 'Footnotes\r\n2003',
       'Footnotes\r\n2004', 'Footnotes\r\n2005', 'Footnotes\r\n2006',
       'Footnotes\r\n2007', 'Footnotes\r\n2008', 'Footnotes\r\n2009',
       'Footnotes\r\n2010', 'Footnotes\r\n2011', 'Footnotes\r\n2012',
       'Footnotes\r\n2013', 'Footnotes\r\n2014', 'Footnotes\r\n2015',
       'Footnotes\r\n2016', 'Footnotes\r\n2017'],
      dtype='object')

In [27]:
fresh_ab.drop(['Footnotes\r\n1990', 'Footnotes\r\n1991',
       'Footnotes\r\n1992', 'Footnotes\r\n1993', 'Footnotes\r\n1994',
       'Footnotes\r\n1995', 'Footnotes\r\n1996', 'Footnotes\r\n1997',
       'Footnotes\r\n1998', 'Footnotes\r\n1999', 'Footnotes\r\n2000',
       'Footnotes\r\n2001', 'Footnotes\r\n2002', 'Footnotes\r\n2003',
       'Footnotes\r\n2004', 'Footnotes\r\n2005', 'Footnotes\r\n2006',
       'Footnotes\r\n2007', 'Footnotes\r\n2008', 'Footnotes\r\n2009',
       'Footnotes\r\n2010', 'Footnotes\r\n2011', 'Footnotes\r\n2012',
       'Footnotes\r\n2013', 'Footnotes\r\n2014', 'Footnotes\r\n2015',
       'Footnotes\r\n2016', 'Footnotes\r\n2017'], axis='columns',
                                        inplace= True)

In [28]:
fresh_ab

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
CountryID,,,,,,,,,,,,,,,,,,,,,
8.0,Albania,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,1123,1194,...,1188
12.0,Algeria,...,...,...,...,...,...,...,...,...,...,6950,7150,7203,7515,7800,4642.759766,4819.819824,5512.660156,7800,7730
20.0,Andorra,...,...,...,...,...,...,...,...,...,...,16.39779282,15.4166193,15.82946682,16.04419327,16.58762169,17.15185165,15.44569111,15.66994095,...,...
28.0,Antigua and Barbuda,3.380000114,...,...,...,...,0.970000029,...,...,...,...,3.529999971,3.950000048,3.789999962,3.75999999,3.809999943,2.769999981,1.080000043,1.25,...,...
51.0,Armenia,3942,...,...,...,...,2331,2077,2250.699951,2144,...,2873.5,2504.699951,2326.399902,2438.300049,2941.199951,2955.100098,2860.199951,3271.699951,3181.899902,2865.399902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887.0,Yemen,...,...,...,...,...,3942,...,...,...,...,...,...,5305,...,...,...,...,5911,...,...
894.0,Zambia,...,...,...,...,...,2431,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716.0,Zimbabwe,33107.64063,...,...,...,...,27045.39453,38103.42188,41901.375,38912.28516,...,52445.32813,59317.16797,60038.69531,60379.23047,49003.97656,52851.39844,49350.78125,37037.32813,44669.73047,48978.59766


In [29]:
ghg

,Country,Latest Year Available,Total GHG emissions without LULUCF (1000 tonnes of CO₂ equivalent),GHG from energy (1000 tonnes of CO₂ equivalent),"GHG from energy, as percentage to total","GHG from energy, of which: from transport (1000 tonnes of CO₂ equivalent)","GHG from energy, of which: from transport, as percentage to total",GHG from industrial processes and product use (1000 tonnes of CO₂ equivalent),"GHG from industrial processes and product use, as percentage to total",GHG from agriculture (1000 tonnes of CO₂ equivalent),"GHG from agriculture, as percentage to total",GHG from waste (1000 tonnes of CO₂ equivalent),"GHG from waste, as percentage to total"
Country ID,,,,,,,,,,,,,
4,Afghanistan,2013,"43,377.00","10,281.00",23.7,"5,217.00",12.0,212.00,0.5,"32,744.00",75.5,140.00,0.3
8,Albania,2009,"8,125.70","4,466.04",55.0,"2,306.46",28.4,"1,701.12",20.9,"1,130.86",13.9,827.68,10.2
12,Algeria,2000,"1,11,022.59","87,595.60",78.9,"12,789.98",11.5,"5,463.83",4.9,"6,534.62",5.9,"11,428.54",10.3
24,Angola,2005,"61,610.76","37,732.06",61.2,...,...,352.00,0.6,"22,575.40",36.6,951.30,1.5
28,Antigua and Barbuda,2000,597.75,372.72,62.4,182.63,30.6,...,...,104.33,17.5,120.70,20.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,Venezuela (Bolivarian Republic of),1999,"1,92,192.24","1,43,561.20",74.7,"33,991.00",17.7,"9,206.20",4.8,"32,959.70",17.1,"6,465.14",3.4
704,Viet Nam,2013,"2,78,441.86","1,47,703.33",53.0,"29,680.72",10.7,"31,767.39",11.4,"81,166.04",29.2,"17,805.10",6.4
887,Yemen,2012,"37,942.87","23,549.08",62.1,"5,739.01",15.1,"1,398.00",3.7,"10,879.79",28.7,"2,116.00",5.6


In [30]:
ghg.isnull().sum()

Country                                                                          0
Latest Year Available                                                            0
Total GHG emissions without LULUCF (1000 tonnes of CO₂ equivalent)               0
GHG from energy (1000 tonnes of CO₂ equivalent)                                  0
GHG from energy, as percentage to total                                          0
GHG from energy, of which: from transport (1000 tonnes of CO₂ equivalent)        0
GHG from energy, of which: from transport, as percentage to total                0
GHG from industrial processes and product use (1000 tonnes of CO₂ equivalent)    0
GHG from industrial processes and product use, as percentage to total            0
GHG from agriculture (1000 tonnes of CO₂ equivalent)                             0
GHG from agriculture, as percentage to total                                     0
GHG from waste (1000 tonnes of CO₂ equivalent)                                   0
GHG 

In [31]:
ghg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 192 entries, 4 to 716
Data columns (total 13 columns):
 #   Column                                                                         Non-Null Count  Dtype 
---  ------                                                                         --------------  ----- 
 0   Country                                                                        192 non-null    object
 1   Latest Year Available                                                          192 non-null    int64 
 2   Total GHG emissions without LULUCF (1000 tonnes of CO₂ equivalent)             192 non-null    object
 3   GHG from energy (1000 tonnes of CO₂ equivalent)                                192 non-null    object
 4   GHG from energy, as percentage to total                                        192 non-null    object
 5   GHG from energy, of which: from transport (1000 tonnes of CO₂ equivalent)      192 non-null    object
 6   GHG from energy, of which: from transpo

In [54]:
url = "https://meteostat.p.rapidapi.com/stations/daily"

# Set stations
stations = ["03969",
            "03962",
            "03955",
            "06660",
            "06700",
            "06601"
            ]

# Set API key (defined already)
headers = {
	"X-RapidAPI-Key": "38c2ec0004msh8992e15e6b32155p15fd43jsn0376b99a111e",
	"X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

# Create empty dataframe, will be used to append each location's weather data
weather_df = pd.DataFrame([])



# Loop through all locations
for station in stations:
    # API call start for open weather
    querystring = {"station":station,"start":"1980-01-01","end":"1989-01-01"}

    response = requests.get(url, headers=headers, params=querystring)
    #API call end

    # time.sleep(1) #uncomment if you run into a query limit
    
    # Decode repsonse with json decoder
    weather_temp = response.json()
    
    # Flatten json response
    weather_temp_df = pd.json_normalize(weather_temp, 
                                        sep="_", 
                                        record_path="data",
                                        meta=["station"], 
                                        record_prefix="weather_",
                                        errors='ignore'
                                        )
    weather_temp_df['station']=station
    # concatenate dataframes
    weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)

# Print final dataset weather_df
weather_df_ir_sw_1=weather_df
weather_df_ir_sw_1.replace(['03969', '03962','03955'], 'Ireland', inplace=True)
weather_df_ir_sw_1.replace(['06660', '06700', '06601'], 'Switzerland', inplace=True)
weather_df_ir_sw_1

/var/folders/rr/xf6qns716mb44gzqznddsss80000gn/T/ipykernel_92937/481875035.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)
/var/folders/rr/xf6qns716mb44gzqznddsss80000gn/T/ipykernel_92937/481875035.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)


,weather_date,weather_tavg,weather_tmin,weather_tmax,weather_prcp,weather_snow,weather_wdir,weather_wspd,weather_wpgt,weather_pres,weather_tsun,station
0,1980-01-01,0.6,-3.7,4.0,0.0,NaN,NaN,NaN,None,NaN,None,Ireland
1,1980-01-02,0.9,-7.1,10.1,2.3,NaN,182.0,11.0,None,1023.0,None,Ireland
2,1980-01-03,10.1,-6.0,11.9,3.7,NaN,208.0,24.8,None,NaN,None,Ireland
3,1980-01-04,5.8,2.9,6.5,0.1,NaN,232.0,26.8,None,999.1,None,Ireland
4,1980-01-05,5.7,3.1,7.3,0.0,NaN,257.0,30.3,None,1005.8,None,Ireland
...,...,...,...,...,...,...,...,...,...,...,...,...
19729,1988-12-28,NaN,1.0,6.2,0.0,0.0,NaN,NaN,None,NaN,None,Switzerland
19730,1988-12-29,NaN,0.6,7.9,0.0,0.0,NaN,NaN,None,NaN,None,Switzerland
19731,1988-12-30,NaN,-2.2,4.7,0.0,0.0,NaN,NaN,None,NaN,None,Switzerland
19732,1988-12-31,NaN,-1.1,0.3,0.0,0.0,NaN,NaN,None,NaN,None,Switzerland


In [55]:
url = "https://meteostat.p.rapidapi.com/stations/daily"

# Set stations
stations = ["03969",
            "03962",
            "03955",
            "06660",
            "06700",
            "06601"
            ]

# Set API key (defined already)
headers = {
	"X-RapidAPI-Key": "38c2ec0004msh8992e15e6b32155p15fd43jsn0376b99a111e",
	"X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

# Create empty dataframe, will be used to append each location's weather data
weather_df = pd.DataFrame([])



# Loop through all locations
for station in stations:
    # API call start for open weather
    querystring = {"station":station,"start":"1989-01-02","end":"1998-01-01"}

    response = requests.get(url, headers=headers, params=querystring)
    #API call end

    # time.sleep(1) #uncomment if you run into a query limit
    
    # Decode repsonse with json decoder
    weather_temp = response.json()
    
    # Flatten json response
    weather_temp_df = pd.json_normalize(weather_temp, 
                                        sep="_", 
                                        record_path="data",
                                        meta=["station"], 
                                        record_prefix="weather_",
                                        errors='ignore'
                                        )
    weather_temp_df['station']=station
    # concatenate dataframes
    weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)

# Print final dataset weather_df
weather_df_ir_sw_2=weather_df
weather_df_ir_sw_2.replace(['03969', '03962','03955'], 'Ireland', inplace=True)
weather_df_ir_sw_2.replace(['06660', '06700', '06601'], 'Switzerland', inplace=True)
weather_df_ir_sw_2

/var/folders/rr/xf6qns716mb44gzqznddsss80000gn/T/ipykernel_92937/1188385840.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)
/var/folders/rr/xf6qns716mb44gzqznddsss80000gn/T/ipykernel_92937/1188385840.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)


,weather_date,weather_tavg,weather_tmin,weather_tmax,weather_prcp,weather_snow,weather_wdir,weather_wspd,weather_wpgt,weather_pres,weather_tsun,station
0,1989-01-02,7.4,3.9,9.1,0.0,NaN,NaN,10.1,None,1031.2,None,Ireland
1,1989-01-03,9.1,5.6,11.9,1.1,NaN,165.0,21.5,None,1018.7,None,Ireland
2,1989-01-04,6.1,1.4,11.0,0.0,NaN,238.0,33.7,None,NaN,None,Ireland
3,1989-01-05,7.6,0.6,11.5,2.6,NaN,239.0,28.6,None,1016.2,None,Ireland
4,1989-01-06,8.2,4.4,9.9,0.1,NaN,288.0,16.6,None,1018.3,None,Ireland
...,...,...,...,...,...,...,...,...,...,...,...,...
19717,1997-12-28,NaN,-0.3,6.0,0.0,0.0,NaN,NaN,None,NaN,None,Switzerland
19718,1997-12-29,NaN,-1.2,5.2,0.0,0.0,NaN,NaN,None,NaN,None,Switzerland
19719,1997-12-30,NaN,-0.8,4.1,0.0,0.0,NaN,NaN,None,NaN,None,Switzerland
19720,1997-12-31,NaN,3.1,5.4,0.7,0.0,NaN,NaN,None,NaN,None,Switzerland


In [56]:
url = "https://meteostat.p.rapidapi.com/stations/daily"

# Set stations
stations = ["03969",
            "03962",
            "03955",
            "06660",
            "06700",
            "06601"
            ]

# Set API key (defined already)
headers = {
	"X-RapidAPI-Key": "38c2ec0004msh8992e15e6b32155p15fd43jsn0376b99a111e",
	"X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

# Create empty dataframe, will be used to append each location's weather data
weather_df = pd.DataFrame([])



# Loop through all locations
for station in stations:
    # API call start for open weather
    querystring = {"station":station,"start":"1998-01-02","end":"2007-01-01"}

    response = requests.get(url, headers=headers, params=querystring)
    #API call end

    # time.sleep(1) #uncomment if you run into a query limit
    
    # Decode repsonse with json decoder
    weather_temp = response.json()
    
    # Flatten json response
    weather_temp_df = pd.json_normalize(weather_temp, 
                                        sep="_", 
                                        record_path="data",
                                        meta=["station"], 
                                        record_prefix="weather_",
                                        errors='ignore'
                                        )
    weather_temp_df['station']=station
    # concatenate dataframes
    weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)

# Print final dataset weather_df
weather_df_ir_sw_3=weather_df
weather_df_ir_sw_3.replace(['03969', '03962','03955'], 'Ireland', inplace=True)
weather_df_ir_sw_3.replace(['06660', '06700', '06601'], 'Switzerland', inplace=True)
weather_df_ir_sw_3

/var/folders/rr/xf6qns716mb44gzqznddsss80000gn/T/ipykernel_92937/1021249776.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)
/var/folders/rr/xf6qns716mb44gzqznddsss80000gn/T/ipykernel_92937/1021249776.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)


,weather_date,weather_tavg,weather_tmin,weather_tmax,weather_prcp,weather_snow,weather_wdir,weather_wspd,weather_wpgt,weather_pres,weather_tsun,station
0,1998-01-02,5.7,3.4,12.4,0.9,NaN,234.0,15.1,None,981.5,None,Ireland
1,1998-01-03,5.7,4.0,6.0,8.8,NaN,NaN,NaN,None,NaN,None,Ireland
2,1998-01-04,3.3,0.5,6.1,2.4,NaN,NaN,NaN,None,NaN,None,Ireland
3,1998-01-05,1.5,-0.4,6.7,1.7,NaN,NaN,NaN,None,NaN,None,Ireland
4,1998-01-06,5.2,-2.2,10.0,0.8,NaN,NaN,NaN,None,NaN,None,Ireland
...,...,...,...,...,...,...,...,...,...,...,...,...
19717,2006-12-28,-0.6,-3.5,4.1,0.0,0.0,NaN,NaN,None,NaN,None,Switzerland
19718,2006-12-29,-0.2,-2.7,3.6,0.0,0.0,NaN,NaN,None,NaN,None,Switzerland
19719,2006-12-30,1.1,-1.6,4.5,0.3,0.0,NaN,NaN,None,NaN,None,Switzerland
19720,2006-12-31,10.1,4.3,14.8,3.8,0.0,NaN,NaN,None,NaN,None,Switzerland


In [60]:
url = "https://meteostat.p.rapidapi.com/stations/daily"

# Set stations
stations = ["03969",
            "03962",
            "03955",
            "06660",
            "06700",
            "06601"
            ]

# Set API key (defined already)
headers = {
	"X-RapidAPI-Key": "38c2ec0004msh8992e15e6b32155p15fd43jsn0376b99a111e",
	"X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

# Create empty dataframe, will be used to append each location's weather data
weather_df = pd.DataFrame([])



# Loop through all locations
for station in stations:
    # API call start for open weather
    querystring = {"station":station,"start":"2007-01-02","end":"2011-01-01"}

    response = requests.get(url, headers=headers, params=querystring)
    #API call end

    # time.sleep(1) #uncomment if you run into a query limit
    
    # Decode repsonse with json decoder
    weather_temp = response.json()
    
    # Flatten json response
    weather_temp_df = pd.json_normalize(weather_temp, 
                                        sep="_", 
                                        record_path="data",
                                        meta=["station"], 
                                        record_prefix="weather_",
                                        errors='ignore'
                                        )
    weather_temp_df['station']=station
    # concatenate dataframes
    weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)

# Print final dataset weather_df
weather_df_ir_sw_4=weather_df
weather_df_ir_sw_4.replace(['03969', '03962','03955'], 'Ireland', inplace=True)
weather_df_ir_sw_4.replace(['06660', '06700', '06601'], 'Switzerland', inplace=True)
weather_df_ir_sw_4

/var/folders/rr/xf6qns716mb44gzqznddsss80000gn/T/ipykernel_92937/2667741573.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)
/var/folders/rr/xf6qns716mb44gzqznddsss80000gn/T/ipykernel_92937/2667741573.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)


,weather_date,weather_tavg,weather_tmin,weather_tmax,weather_prcp,weather_snow,weather_wdir,weather_wspd,weather_wpgt,weather_pres,weather_tsun,station
0,2007-01-02,6.4,3.1,8.8,NaN,NaN,248.0,30.2,None,1023.8,None,Ireland
1,2007-01-03,10.8,6.7,12.5,1.0,NaN,224.0,35.9,None,1009.4,None,Ireland
2,2007-01-04,8.1,7.6,9.5,2.5,NaN,256.0,36.5,None,NaN,None,Ireland
3,2007-01-05,8.2,6.1,10.5,NaN,NaN,233.0,26.1,None,1013.0,None,Ireland
4,2007-01-06,7.4,5.5,9.8,0.0,NaN,220.0,22.1,None,1009.3,None,Ireland
...,...,...,...,...,...,...,...,...,...,...,...,...
8761,2010-12-28,-1.8,-3.8,0.3,0.8,130.0,NaN,NaN,None,NaN,None,Switzerland
8762,2010-12-29,0.7,-0.1,2.0,0.0,120.0,NaN,NaN,None,NaN,None,Switzerland
8763,2010-12-30,0.1,-0.4,0.6,0.0,80.0,NaN,NaN,None,NaN,None,Switzerland
8764,2010-12-31,0.2,-0.5,0.9,0.0,60.0,NaN,NaN,None,NaN,None,Switzerland


In [61]:
url = "https://meteostat.p.rapidapi.com/stations/daily"

# Set stations
stations = ["03969",
            "03962",
            "03955",
            "06660",
            "06700",
            "06601"
            ]

# Set API key (defined already)
headers = {
	"X-RapidAPI-Key": "38c2ec0004msh8992e15e6b32155p15fd43jsn0376b99a111e",
	"X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

# Create empty dataframe, will be used to append each location's weather data
weather_df = pd.DataFrame([])



# Loop through all locations
for station in stations:
    # API call start for open weather
    querystring = {"station":station,"start":"2011-01-02","end":"2019-12-31"}

    response = requests.get(url, headers=headers, params=querystring)
    #API call end

    # time.sleep(1) #uncomment if you run into a query limit
    
    # Decode repsonse with json decoder
    weather_temp = response.json()
    
    # Flatten json response
    weather_temp_df = pd.json_normalize(weather_temp, 
                                        sep="_", 
                                        record_path="data",
                                        meta=["station"], 
                                        record_prefix="weather_",
                                        errors='ignore'
                                        )
    weather_temp_df['station']=station
    # concatenate dataframes
    weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)

# Print final dataset weather_df
weather_df_ir_sw_5=weather_df
weather_df_ir_sw_5.replace(['03969', '03962','03955'], 'Ireland', inplace=True)
weather_df_ir_sw_5.replace(['06660', '06700', '06601'], 'Switzerland', inplace=True)
weather_df_ir_sw_5

,weather_date,weather_tavg,weather_tmin,weather_tmax,weather_prcp,weather_snow,weather_wdir,weather_wspd,weather_wpgt,weather_pres,weather_tsun,station
0,2011-01-02,1.7,0.0,3.0,NaN,NaN,15.0,11.9,NaN,1030.6,None,Ireland
1,2011-01-03,0.5,-0.4,2.7,NaN,NaN,268.0,12.3,NaN,1024.0,None,Ireland
2,2011-01-04,3.5,-0.3,5.3,0.0,NaN,232.0,30.3,NaN,1007.4,None,Ireland
3,2011-01-05,3.6,1.2,5.2,NaN,NaN,245.0,26.7,NaN,993.8,None,Ireland
4,2011-01-06,0.2,-5.9,3.4,0.0,NaN,308.0,15.0,NaN,997.6,None,Ireland
...,...,...,...,...,...,...,...,...,...,...,...,...
19711,2019-12-27,6.9,5.6,7.9,5.2,0.0,198.0,5.2,16.7,1027.1,None,Switzerland
19712,2019-12-28,3.9,-0.6,5.7,0.0,0.0,358.0,5.4,20.4,1037.8,None,Switzerland
19713,2019-12-29,-0.8,-2.7,2.6,0.0,0.0,70.0,5.6,18.5,1039.3,None,Switzerland
19714,2019-12-30,0.6,-3.0,5.1,0.0,0.0,99.0,5.8,16.7,1036.4,None,Switzerland


In [66]:
# Concatenate the DataFrames vertically
weather_df_ir_sw = pd.concat([weather_df_ir_sw_1, weather_df_ir_sw_2, weather_df_ir_sw_3, weather_df_ir_sw_4, weather_df_ir_sw_5], ignore_index=True)

# Display the merged table
weather_df_ir_sw

/var/folders/rr/xf6qns716mb44gzqznddsss80000gn/T/ipykernel_92937/1248764182.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_df_ir_sw = pd.concat([weather_df_ir_sw_1, weather_df_ir_sw_2, weather_df_ir_sw_3, weather_df_ir_sw_4, weather_df_ir_sw_5], ignore_index=True)


,weather_date,weather_tavg,weather_tmin,weather_tmax,weather_prcp,weather_snow,weather_wdir,weather_wspd,weather_wpgt,weather_pres,weather_tsun,station
0,1980-01-01,0.6,-3.7,4.0,0.0,NaN,NaN,NaN,NaN,NaN,None,Ireland
1,1980-01-02,0.9,-7.1,10.1,2.3,NaN,182.0,11.0,NaN,1023.0,None,Ireland
2,1980-01-03,10.1,-6.0,11.9,3.7,NaN,208.0,24.8,NaN,NaN,None,Ireland
3,1980-01-04,5.8,2.9,6.5,0.1,NaN,232.0,26.8,NaN,999.1,None,Ireland
4,1980-01-05,5.7,3.1,7.3,0.0,NaN,257.0,30.3,NaN,1005.8,None,Ireland
...,...,...,...,...,...,...,...,...,...,...,...,...
87655,2019-12-27,6.9,5.6,7.9,5.2,0.0,198.0,5.2,16.7,1027.1,None,Switzerland
87656,2019-12-28,3.9,-0.6,5.7,0.0,0.0,358.0,5.4,20.4,1037.8,None,Switzerland
87657,2019-12-29,-0.8,-2.7,2.6,0.0,0.0,70.0,5.6,18.5,1039.3,None,Switzerland
87658,2019-12-30,0.6,-3.0,5.1,0.0,0.0,99.0,5.8,16.7,1036.4,None,Switzerland


In [67]:
weather_df_ir_sw.drop(['weather_wpgt', 'weather_pres','weather_tsun','weather_snow'], axis='columns',inplace= True)

In [68]:
#converting the datatype of the weather date to datetime
weather_df_ir_sw['weather_date'] = weather_df_ir_sw['weather_date'].astype('datetime64[ns]')

In [72]:
# the dataset should be uploaded to the schema 'capstone_hydrogenious'


# Import get_engine function from sql_functions_anja.py and set it to a variable called engine

from sql_functions_anja import get_engine
engine=get_engine()

# Import psycopg2
#import psycopg2

if engine!=None:
    try:
        weather_df_ir_sw.to_sql(name='weather_ir_sw', # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema='capstone_hydrogenious', # your class schema
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
    else:
        print('no engine')

The table was imported successfully.
no engine


In [77]:
weather_df_ir_sw['year'] = weather_df_ir_sw['weather_date'].dt.year
weather_df_ir_sw

,weather_date,weather_tavg,weather_tmin,weather_tmax,weather_prcp,weather_wdir,weather_wspd,station,year
0,1980-01-01,0.6,-3.7,4.0,0.0,NaN,NaN,Ireland,1980
1,1980-01-02,0.9,-7.1,10.1,2.3,182.0,11.0,Ireland,1980
2,1980-01-03,10.1,-6.0,11.9,3.7,208.0,24.8,Ireland,1980
3,1980-01-04,5.8,2.9,6.5,0.1,232.0,26.8,Ireland,1980
4,1980-01-05,5.7,3.1,7.3,0.0,257.0,30.3,Ireland,1980
...,...,...,...,...,...,...,...,...,...
87655,2019-12-27,6.9,5.6,7.9,5.2,198.0,5.2,Switzerland,2019
87656,2019-12-28,3.9,-0.6,5.7,0.0,358.0,5.4,Switzerland,2019
87657,2019-12-29,-0.8,-2.7,2.6,0.0,70.0,5.6,Switzerland,2019
87658,2019-12-30,0.6,-3.0,5.1,0.0,99.0,5.8,Switzerland,2019


In [109]:
#starts the process for location data
weather_df_ir_sw['lon'] = 0 
weather_df_ir_sw['lat'] = 0
weather_df_ir_sw

,weather_date,weather_tavg,weather_tmin,weather_tmax,weather_prcp,weather_wdir,weather_wspd,station,year,lon,lat
0,1980-01-01,0.6,-3.7,4.0,0.0,NaN,NaN,Ireland,1980,0,0
1,1980-01-02,0.9,-7.1,10.1,2.3,182.0,11.0,Ireland,1980,0,0
2,1980-01-03,10.1,-6.0,11.9,3.7,208.0,24.8,Ireland,1980,0,0
3,1980-01-04,5.8,2.9,6.5,0.1,232.0,26.8,Ireland,1980,0,0
4,1980-01-05,5.7,3.1,7.3,0.0,257.0,30.3,Ireland,1980,0,0
...,...,...,...,...,...,...,...,...,...,...,...
87655,2019-12-27,6.9,5.6,7.9,5.2,198.0,5.2,Switzerland,2019,0,0
87656,2019-12-28,3.9,-0.6,5.7,0.0,358.0,5.4,Switzerland,2019,0,0
87657,2019-12-29,-0.8,-2.7,2.6,0.0,70.0,5.6,Switzerland,2019,0,0
87658,2019-12-30,0.6,-3.0,5.1,0.0,99.0,5.8,Switzerland,2019,0,0


In [140]:
for i in range(87660):  # Assuming you want to iterate from 0 to 87660

    if weather_df_ir_sw.loc[i, 'station'] == 'Ireland':
        weather_df_ir_sw.loc[i, 'lon'] = -7.3055 #west and south need minus
        weather_df_ir_sw.loc[i, 'lat'] = 53.7798
    else:
        weather_df_ir_sw.loc[i, 'lon'] = 8.2275
        weather_df_ir_sw.loc[i, 'lat'] = 46.8182

In [141]:
weather_df_ir_sw

,weather_date,weather_tavg,weather_tmin,weather_tmax,weather_prcp,weather_wdir,weather_wspd,station,year,lon,lat
0,1980-01-01,0.6,-3.7,4.0,0.0,NaN,NaN,Ireland,1980,-7.3055,53.7798
1,1980-01-02,0.9,-7.1,10.1,2.3,182.0,11.0,Ireland,1980,-7.3055,53.7798
2,1980-01-03,10.1,-6.0,11.9,3.7,208.0,24.8,Ireland,1980,-7.3055,53.7798
3,1980-01-04,5.8,2.9,6.5,0.1,232.0,26.8,Ireland,1980,-7.3055,53.7798
4,1980-01-05,5.7,3.1,7.3,0.0,257.0,30.3,Ireland,1980,-7.3055,53.7798
...,...,...,...,...,...,...,...,...,...,...,...
87655,2019-12-27,6.9,5.6,7.9,5.2,198.0,5.2,Switzerland,2019,8.2275,46.8182
87656,2019-12-28,3.9,-0.6,5.7,0.0,358.0,5.4,Switzerland,2019,8.2275,46.8182
87657,2019-12-29,-0.8,-2.7,2.6,0.0,70.0,5.6,Switzerland,2019,8.2275,46.8182
87658,2019-12-30,0.6,-3.0,5.1,0.0,99.0,5.8,Switzerland,2019,8.2275,46.8182


In [142]:
agg_dict = {'weather_tavg':['mean'], 'weather_tmin':['mean'],
            'weather_tmax':['mean'],'weather_prcp':['sum', 'max', 'min', 'mean'],
            'weather_wdir':['mean'], 'weather_wspd':['mean'], 'lon':['mean'], 'lat':['mean']}

weather_df_ir_sw_y = weather_df_ir_sw.groupby(['year', 'station'], as_index=False).agg(agg_dict)

In [143]:
weather_df_ir_sw_y

year      station weather_tavg weather_tmin weather_tmax weather_prcp  \
                              mean         mean         mean          sum   
0   1980      Ireland     9.633607     6.366917    12.861341       3092.5   
1   1980  Switzerland     8.555464     5.144900    12.903552       2866.3   
2   1981      Ireland     9.837352     6.656787    13.115978       2918.1   
3   1981  Switzerland     9.199312     5.286484    13.756438       3254.7   
4   1982      Ireland    10.021461     6.529163    13.473346       3231.2   
..   ...          ...          ...          ...          ...          ...   
75  2017  Switzerland    10.989406     6.764865    16.044671       2492.0   
76  2018      Ireland    10.299269     6.990959    13.907770       2188.7   
77  2018  Switzerland    11.934521     7.589132    16.595616       2459.2   
78  2019      Ireland    10.224749     7.188402    13.603568       2026.7   
79  2019  Switzerland    11.335068     6.888402    16.097078       2669.2   

                        weather_wdir weather_wspd     lon      lat  
     max  min      mean         mean         mean    mean     mean  
0   42.9  0.0  2.876744   214.134431    18.146970 -7.3055  53.7798  
1   42.9  0.0  2.610474   150.740000     9.399216  8.2275  46.8182  
2   52.1  0.0  2.694460   220.720663    18.809576 -7.3055  53.7798  
3   58.9  0.0  2.972329   164.625000     9.831321  8.2275  46.8182  
4   45.0  0.0  3.383455   216.160819    18.549260 -7.3055  53.7798  
..   ...  ...       ...          ...          ...     ...      ...  
75  48.3  0.0  2.275799   184.333333     8.811570  8.2275  46.8182  
76  32.0  0.0  2.574941   197.581118    18.839103 -7.3055  53.7798  
77  54.4  0.0  2.245845   154.492754     7.651012  8.2275  46.8182  
78  37.1  0.0  2.735088   212.939560    18.481593 -7.3055  53.7798  
79  48.0  0.0  2.437626   171.112339     7.920513  8.2275  46.8182  

[80 rows x 13 columns]

In [144]:
# the dataset should be uploaded to the schema 'capstone_hydrogenious'


# Import get_engine function from sql_functions_anja.py and set it to a variable called engine

from sql_functions_anja import get_engine
engine=get_engine()

# Import psycopg2
#import psycopg2

if engine!=None:
    try:
        weather_df_ir_sw_y.to_sql(name='weather_ir_sw_y', # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema='capstone_hydrogenious', # your class schema
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
    else:
        print('no engine')



#Ends the process for location data

The table was imported successfully.
no engine


In [36]:
from sql_functions_anja import get_dataframe

sql_query=f'select * from capstone_hydrogenious.crops_all_data;'
df_crops_all=get_dataframe(sql_query)

In [37]:
df_crops_all

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,198,Slovenia,534,Peaches and nectarines,5419,Yield,2009,2009,hg/ha,174420.0,Fc
1,198,Slovenia,534,Peaches and nectarines,5419,Yield,2010,2010,hg/ha,156697.0,Fc
2,198,Slovenia,534,Peaches and nectarines,5419,Yield,2011,2011,hg/ha,163098.0,Fc
3,198,Slovenia,534,Peaches and nectarines,5419,Yield,2012,2012,hg/ha,135956.0,Fc
4,198,Slovenia,534,Peaches and nectarines,5419,Yield,2013,2013,hg/ha,141807.0,Fc
...,...,...,...,...,...,...,...,...,...,...,...
2513863,198,Slovenia,534,Peaches and nectarines,5419,Yield,2005,2005,hg/ha,194106.0,Fc
2513864,198,Slovenia,534,Peaches and nectarines,5419,Yield,2006,2006,hg/ha,158476.0,Fc
2513865,198,Slovenia,534,Peaches and nectarines,5419,Yield,2007,2007,hg/ha,157602.0,Fc
2513866,198,Slovenia,534,Peaches and nectarines,5419,Yield,2008,2008,hg/ha,112573.0,Fc


In [38]:
df_crops_all.isnull().sum()

Area Code            0
Area                 0
Item Code            0
Item                 0
Element Code         0
Element              0
Year Code            0
Year                 0
Unit                 0
Value           154312
Flag            530383
dtype: int64

In [39]:
df_crops_ir_sw = df_crops_all.query("(`Area` == 'Ireland') or (`Area` == 'Switzerland')")
df_crops_ir_sw

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
83770,211,Switzerland,515,Apples,5312,Area harvested,1961,1961,ha,2700.0,F
83771,211,Switzerland,515,Apples,5312,Area harvested,1962,1962,ha,4800.0,F
83772,211,Switzerland,515,Apples,5312,Area harvested,1963,1963,ha,3600.0,F
83773,211,Switzerland,515,Apples,5312,Area harvested,1964,1964,ha,4000.0,F
83774,211,Switzerland,515,Apples,5312,Area harvested,1965,1965,ha,2478.0,None
...,...,...,...,...,...,...,...,...,...,...,...
1876036,104,Ireland,1735,Vegetables Primary,5510,Production,2015,2015,tonnes,222293.0,A
1876037,104,Ireland,1735,Vegetables Primary,5510,Production,2016,2016,tonnes,231989.0,A
1876038,104,Ireland,1735,Vegetables Primary,5510,Production,2017,2017,tonnes,204672.0,A
1876039,104,Ireland,1735,Vegetables Primary,5510,Production,2018,2018,tonnes,170700.0,A


In [71]:
# the dataset should be uploaded to the schema 'capstone_hydrogenious'


# Import get_engine function from sql_functions_anja.py and set it to a variable called engine
#from sql_functions_anja import get_engine

engine=get_engine()

# Import psycopg2
#import psycopg2

if engine!=None:
    try:
        df_crops_ir_sw.to_sql(name='crops_ir_sw', # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema='capstone_hydrogenious', # your class schema
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
    else:
        print('no engine')

The table was imported successfully.
no engine


In [103]:

ireland_crops_pro = df_crops_all[
    (df_crops_all['Area'] == 'Ireland') &
    (df_crops_all['Element'] == 'Production') &
    (df_crops_all['Year'].between(1980, 2019))
]

crop_grouped_pro_ir = ireland_crops_pro.groupby('Item')['Value'].sum().reset_index()

# Sort the data in descending order based on total production
sorted_crops_pro_ir = crop_grouped_pro_ir.sort_values(by='Value', ascending=False)

top_10_crops_pro_ir = sorted_crops_pro_ir.head(10)

top_10_crops_pro_ir

,Item,Value
15,"Cereals, Total",84598687.0
5,Barley,53352271.0
69,Sugar beet,39556866.0
68,Sugar Crops Primary,39556866.0
81,Wheat,25490519.0
62,"Roots and Tubers, Total",21024205.0
54,Potatoes,21024205.0
76,Vegetables Primary,9197093.0
43,Oats,5598139.0
77,"Vegetables, fresh nes",2029765.0


In [146]:
ireland_crops_pro = df_crops_all[
    (df_crops_all['Area'] == 'Ireland') &
    (df_crops_all['Element'] == 'Yield') &
    (df_crops_all['Year'].between(1980, 2019))
]

crop_grouped_pro_ir = ireland_crops_pro.groupby('Item')['Value'].sum().reset_index()

# Sort the data in descending order based on total production
sorted_crops_pro_ir = crop_grouped_pro_ir.sort_values(by='Value', ascending=False)

top_10_crops_pro_ir = sorted_crops_pro_ir.head(10)

top_10_crops_pro_ir

,Item,Value
38,Tomatoes,106201379.0
11,Cucumbers and gherkins,68584032.0
36,Sugar beet,17870643.0
35,Sugar Crops Primary,17870643.0
7,Carrots and turnips,17080476.0
39,Vegetables Primary,13820180.0
6,Cabbages and other brassicas,12268986.0
26,Potatoes,12121407.0
31,"Roots and Tubers, Total",12121407.0
23,"Onions, dry",10599898.0


In [104]:
switzerland_crops_pro = df_crops_all[
    (df_crops_all['Area'] == 'Switzerland') &
    (df_crops_all['Element'] == 'Production') &
    (df_crops_all['Year'].between(1980, 2019))
]

crop_grouped_pro_sw = switzerland_crops_pro.groupby('Item')['Value'].sum().reset_index()

# Sort the data in descending order based on total production
sorted_crops_pro_sw = crop_grouped_pro_sw.sort_values(by='Value', ascending=False)

top_10_crops_pro_sw = sorted_crops_pro_sw.head(10)

top_10_crops_pro_sw

,Item,Value
61,Sugar beet,47391317.0
60,Sugar Crops Primary,47391317.0
14,"Cereals, Total",41969412.0
55,"Roots and Tubers, Total",24287725.0
49,Potatoes,24287725.0
20,Fruit Primary,23104917.0
71,Wheat,21128364.0
67,Vegetables Primary,13743786.0
0,Apples,11352340.0
4,Barley,9972046.0
